In [192]:
# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from zipfile import ZipFile
from urllib.request import urlretrieve

%matplotlib inline

In [193]:
import os
from zipfile import ZipFile

def extract_zip(zip_path, extraction_path):
    print(f"Extracting zip file....", end="")

    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(zip_path) as z:
            # Extract ZIP file contents to the specified directory.
            z.extractall(extraction_path)

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)

In [194]:
asset_zip_path = os.path.join(os.getcwd(), "securityCamVideos.zip")
output_directory = os.path.join(os.getcwd(), "securityCamVideos")

# output folder
os.makedirs(output_directory, exist_ok=True)

# Call the extract_zip function
extract_zip(asset_zip_path, output_directory)


Extracting zip file....Done


In [195]:
classFile  = "coco_class_labels.txt"

In [196]:
with open(classFile) as fp:
    labels = fp.read().split("\n")

print(labels)

['unlabeled', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush', '']


In [197]:
modelFile  = os.path.join("models", "ssd_mobilenet_v2_coco_2018_03_29", "frozen_inference_graph.pb")
configFile = os.path.join("models", "ssd_mobilenet_v2_coco_2018_03_29.pbtxt")

In [198]:
# Read the Tensorflow network
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [199]:

def detect_objects_in_video(video_path, net, labels, dim=300):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = None

    person_detected = False  # als er een persoon is of niet

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        objects = detect_objects(net, frame, dim)

        # detectie
        for detection in objects[0,0,:,:]:
            score = float(detection[2])
            class_id = int(detection[1])
            if score > 0.5 and labels[class_id] == 'person':
                if not person_detected:
                    person_detected = True
                    out = cv2.VideoWriter(video_path.replace('.avi', '_detected.avi'), fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

                left = int(detection[3] * frame.shape[1])
                top = int(detection[4] * frame.shape[0])
                right = int(detection[5] * frame.shape[1])
                bottom = int(detection[6] * frame.shape[0])

                # kader rond persoon
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 255), thickness=10)
                label = f"{labels[class_id]}: {score:.2f}"
                cv2.putText(frame, label, (left, top-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

        if person_detected:
            out.write(frame)

    cap.release()
    if out is not None:
        out.release()

In [ ]:
video_directory = r"C:\Users\Bryan\Documents\EHB\AI for robotics\securityCamVideos"
for video_file in os.listdir(video_directory):
    video_path = os.path.join(video_directory, video_file)
    if video_path.endswith(".avi"):
        print(f"Processing {video_file}...")
        detect_objects_in_video(video_path, net, labels)

Processing 202402271732.avi...
Processing 202403011343.avi...
Processing 202403011345.avi...
Processing 202403011346.avi...
Processing 202403011347.avi...
Processing 202403011348.avi...
Processing 202403011349.avi...
Processing 202403011350.avi...
Processing 202403011351.avi...
Processing 202403011352.avi...
Processing 202403011353.avi...
Processing 202403011354.avi...
Processing 202403011355.avi...
Processing 202403011356.avi...
Processing 202403011357.avi...
Processing 202403011358.avi...
Processing 202403011359.avi...
Processing 202403011409.avi...
Processing 202403011415.avi...
Processing 202403011416.avi...
Processing 202403011417.avi...
Processing 202403011418.avi...
Processing 202403011419.avi...
Processing 202403011420.avi...
Processing 202403011421.avi...
Processing 202403011425.avi...
Processing 202403011426.avi...
Processing 202403011427.avi...
Processing 202403011429.avi...
Processing 202403011447.avi...
Processing 202403011449.avi...
Processing 202403011451.avi...
Processi